# Apply subject-to-template transforms to afids
Applying ANTs-generated transforms to Slicer-generated .fcsv files
## Preparations
### Imports

In [2]:
import os
import pandas as pd
from registration_utils import transform_fcsv

### Define paths

In [9]:
# Macaca singe-subjects directory
single_macaca = '/home/niko/MRI/afids/single_macaca'

# Table with subjects
subj_table_csv = os.path.join(single_macaca, 'macaque_subjects.csv')
subj_table = pd.read_csv(subj_table_csv)
# List of subject IDs
subjects = subj_table['subject'].values
print(f'Subjects: {subjects}')

# Subject-to-template (NMTv2.0asym) warps folders
# A. Registration preceded be setting the origin to PMJ
warps_whole = os.path.join(single_macaca, 'registered')
# B. Registration preceded by brain extraction (U-NET model)
warps_brain = os.path.join(single_macaca, 'registered-brain')

# Get afids-macaca root directory
root = os.path.dirname(os.getcwd())

# Folder with MEAN PHASE2 post-QC .fcsv files
# (1 set of consensus afids per subjects)
afids_inputs_means = os.path.join(root, 'data', 'PHASE2_output_afid_postQC_for_PHASE3')

# Folder with ALL PHASE2 post-QC .fcsv files
# (from all raters and sessions, total 96)
afids_inputs_all = os.path.join(root, 'data', 'PHASE2_input_afid_postQC')

# Folders for storing output PHASE 3 .fcsv files
afids_outputs = os.path.join(root, 'data', 'PHASE3_RheMAP_transformed')
# for outputs transformations calculated based on whole head with origin reset to PMJ
afids_outputs_PMJ = os.path.join(afids_outputs, 'origin-PMJ')
# for outputs transformations calculated after brain extracted
afids_outputs_brain = os.path.join(afids_outputs, 'brain-extraction')
# List of both folders
out_folders = [afids_outputs_PMJ, afids_outputs_brain]

Subjects: ['sub-032104' 'sub-032105' 'sub-032107' 'sub-032108' 'sub-032198'
 'sub-032199' 'sub-032201' 'sub-032203' 'sub-032209' 'sub-032210'
 'sub-032211' 'sub-032212' 'sub-032213' 'sub-032214' 'sub-032215'
 'sub-032216']


## Apply transforms to points
The transforms are run iteratively for each of 16 subjects.
* First, the MEAN QC .fcsv files (1 per subject) are transformed
* Second, ALL QC .fcsv files (96, all raters and sessions) are transformed
  
**Important note**: ANTs transforms on points work in the opposite direction (as compared to image transforms).

For example, let's imagine we have to go from subject A to template B either using a linear transform (.mat) or a non-linear warp(.nii.gz)
* Linear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B.mat`
* Linear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B -t [A-to-B.mat, 1]` (inverting the forward transform)
* Nonlinear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B_1Warp.nii.gz` (for RheMAP we would use the `A-to-B_CompositeWarp.nii.gz`)
* Nonlinear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B A-to-B_1InverseWarp.nii.gz` (for RheMAP we would use the `B-to-A_CompositeWarp.nii.gz`)

The application of the `antsApplyTransformsToPoints` command to Slicer-generated .fcsv files is implemented in the `transform_fcsv` function (imported from `registration_utils.py`). The function takes the following arguments:
1. input fcsv: path to input Slicer fcsv file
2. ouput fcsv: path to output Slicer fcsv file
3. transform: path to ANTs transform file (either linear .mat or .nii.gz warp)
4. invert: if 1, linear .mat is inverted

In [13]:
# Iterate over subjects
for subj in subjects:

    print(f'Transforming fiducials between {subj} and NMTv2.0asym')    
    
    # Iterate over warp folders (calculated with or without pre-brain extraction)
    for i, warps in enumerate([warps_whole, warps_brain]):
        # set prefix (with or without '_brain')
        prefix = f'{subj}_brain' if i == 1 else subj
        # Fetch appropriate warp files
        warp_linear = os.path.join(warps, f'{prefix}_to_NMTv2.0asym_Affine_0GenericAffine.mat')
        warp_final = os.path.join(warps, f'{prefix}_to_NMTv2.0asym_CompositeWarp.nii.gz')
        warp_final_inv = os.path.join(warps, f'{prefix}_to_NMTv2.0asym_InverseCompositeWarp.nii.gz')
        # Set output subfolder according to warp calculation type
        out_folder = out_folders[i]

        ###############################################
        # Transform subject MEAN QC consensus file
        ###############################################
        in_subj_mean = os.path.join(afids_inputs_means, f'{subj}_MEAN.fcsv')

        # Run subject-to-template transforms
        # linear
        transform_fcsv(in_subj_mean,
                       os.path.join(out_folder, 'MEAN', f'{subj}_MEAN_in_NMTv2.0asym_linear.fcsv'),
                       warp_linear, invert=1)
        # final
        transform_fcsv(in_subj_mean,
                       os.path.join(out_folder, 'MEAN', f'{subj}_MEAN_in_NMTv2.0asym_final.fcsv'),
                       warp_final_inv, invert=0)

        ###############################################
        # Transform ALL QC .fcsv files for this subject
        ###############################################
        in_subj_all = [os.path.join(afids_inputs_all, f) 
                       for f in os.listdir(afids_inputs_all) if subj in f]
        
        for in_subj in in_subj_all:
            # Get ID for fid32 file
            fid32_id = '_'.join(os.path.basename(in_subj).split('.')[0].split('_')[1:])
            # Run subject-to-template transforms
            # linear
            transform_fcsv(in_subj,
                        os.path.join(out_folder, 'ALL', f'{fid32_id}_in_NMTv2.0asym_linear.fcsv'),
                        warp_linear, invert=1)
            # final
            transform_fcsv(in_subj,
                        os.path.join(out_folder, 'ALL', f'{fid32_id}_in_NMTv2.0asym_final.fcsv'),
                        warp_final_inv, invert=0)

Transforming fiducials between sub-032104 and NMTv2.0asym
Transforming fiducials between sub-032105 and NMTv2.0asym
Transforming fiducials between sub-032107 and NMTv2.0asym
Transforming fiducials between sub-032108 and NMTv2.0asym
Transforming fiducials between sub-032198 and NMTv2.0asym
Transforming fiducials between sub-032199 and NMTv2.0asym
Transforming fiducials between sub-032201 and NMTv2.0asym
Transforming fiducials between sub-032203 and NMTv2.0asym
Transforming fiducials between sub-032209 and NMTv2.0asym
Transforming fiducials between sub-032210 and NMTv2.0asym
Transforming fiducials between sub-032211 and NMTv2.0asym
Transforming fiducials between sub-032212 and NMTv2.0asym
Transforming fiducials between sub-032213 and NMTv2.0asym
Transforming fiducials between sub-032214 and NMTv2.0asym
Transforming fiducials between sub-032215 and NMTv2.0asym
Transforming fiducials between sub-032216 and NMTv2.0asym
